In [ ]:
import pyannote.core
from pydub import AudioSegment
file = {'uri':'1043_meal', 'audio':'/Users/andrei-macpro/Documents/Data/Audio/1043_meal.wav'}
import torch
# speech activity detection model trained on AMI training set
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
# obtain raw SAD scores (as `pyannote.core.SlidingWindowFeature` instance)
sad_scores = sad(file)

In [ ]:
from pyannote.audio.utils.signal import Binarize
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True, 
                    min_duration_off=0.1, min_duration_on=0.1)

# speech regions (as `pyannote.core.Timeline` instance)
speech = binarize.apply(sad_scores, dimension=1)

In [ ]:
# convert timestamps to miliseconds 
type(speech[0])
# speech is a timeline made of segments

In [ ]:
#extract speech and non-speech segments (start and end time) as a dictionary
timestamps_speech=dict(speech)
timestamps_silence = dict(speech.gaps())

In [ ]:
file=AudioSegment.from_wav('/Users/andrei-macpro/Documents/Data/Audio/1043_meal.wav')

In [ ]:
# pydub works in miliseconds so we're going to convert the timestamps from the pyannote seconds to miliseconds
start_times_speech=[x*1000 for x in list(timestamps_speech.keys())]
end_times_speech=[x*1000 for x in list(timestamps_speech.values())]
start_times_silence=[x*1000 for x in list(timestamps_silence.keys())]
end_times_silence=[x*1000 for x in list(timestamps_silence.values())]

In [ ]:
speech_segments=[file[start_time:end_time] for start_time,end_time in zip(start_times_speech, end_times_speech)]
silence_segments=[file[start_time:end_time] for start_time,end_time in zip(start_times_silence, end_times_silence)]

In [ ]:
# export the concatenated speech and silence segments to disk
sum(speech_segments).export('/Users/andrei-macpro/Documents/Data/Audio/speech.wav', format="wav")
sum(silence_segments).export('/Users/andrei-macpro/Documents/Data/Audio/non-speech.wav', format="wav")